# Definicje funkcji pomocniczych

In [ ]:
"""Function definitions that are used in LSB steganography."""
from matplotlib import pyplot as plt
import numpy as np
import binascii
import cv2 as cv
import math
from lorem_text import lorem
plt.rcParams["figure.figsize"] = (18,10)


def encode_as_binary_array(msg):
    """Encode a message as a binary string."""
    msg = msg.encode("utf-8")
    msg = msg.hex()
    msg = [msg[i:i + 2] for i in range(0, len(msg), 2)]
    msg = [ "{:08b}".format(int(el, base=16)) for el in msg]
    return "".join(msg)


def decode_from_binary_array(array):
    """Decode a binary string to utf8."""
    array = [array[i:i+8] for i in range(0, len(array), 8)]
    if len(array[-1]) != 8:
        array[-1] = array[-1] + "0" * (8 - len(array[-1]))
    array = [ "{:02x}".format(int(el, 2)) for el in array]
    array = "".join(array)
    result = binascii.unhexlify(array)
    return result.decode("utf-8", errors="replace")


def load_image(path, pad=False):
    """Load an image.
    
    If pad is set then pad an image to multiple of 8 pixels.
    """
    image = cv.imread(path)
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    if pad:
        y_pad = 8 - (image.shape[0] % 8)
        x_pad = 8 - (image.shape[1] % 8)
        image = np.pad(
            image, ((0, y_pad), (0, x_pad) ,(0, 0)), mode='constant')
    return image


def save_image(path, image):
    """Save an image."""
    plt.imsave(path, image) 


def clamp(n, minn, maxn):
    """Clamp the n value to be in range (minn, maxn)."""
    return max(min(maxn, n), minn)


def hide_message(image, message, nbits=1):
    """Hide a message in an image (LSB).
    
    nbits: number of least significant bits
    """
    nbits = clamp(nbits, 1, 8)
    shape = image.shape
    image = np.copy(image).flatten()
    if len(message) > len(image) * nbits:
        raise ValueError("Message is to long :(")
    
    chunks = [message[i:i + nbits] for i in range(0, len(message), nbits)]
    for i, chunk in enumerate(chunks):
        byte = "{:08b}".format(image[i])
        new_byte = byte[:-nbits] + chunk
        image[i] = int(new_byte, 2)
        
    return image.reshape(shape)


def reveal_message(image, nbits=1, length=0):
    """Reveal the hidden message.
    
    nbits: number of least significant bits
    length: length of the message in bits.
    """
    nbits = clamp(nbits, 1, 8)
    shape = image.shape
    image = np.copy(image).flatten()
    length_in_pixels = math.ceil(length/nbits)
    if len(image) < length_in_pixels or length_in_pixels <= 0:
        length_in_pixels = len(image)
    
    message = ""
    i = 0
    while i < length_in_pixels:
        byte = "{:08b}".format(image[i])
        message += byte[-nbits:]
        i += 1
        
    mod = length % -nbits
    if mod != 0:
        message = message[:mod]
    return message

# Zadanie 1

In [ ]:
image = load_image("images/stiv_jobs.png")
message = "Ja Bombas ja Bombas jak mnie slyszysz?"
n=1

image_with_message = hide_message(image, encode_as_binary_array(message), n)
save_image("images/kmwtw.png", image_with_message)
unknown_image = load_image("images/kmwtw.png")
unknown_message = decode_from_binary_array(reveal_message(unknown_image, nbits=n, length=len(message)*8))
print(unknown_message)

# Zadanie 2

In [ ]:
def mse(imageA, imageB):
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1] * imageA.shape[2])

    return err

message2 = lorem.words(image.shape[0]*image.shape[1]*image.shape[2] // (8*8))
print('Message length (characters):', len(message2), '    Image size:', image.shape)

images_with_message = [hide_message(image, encode_as_binary_array(message2), x) for x in range(1,9)]

f, ar = plt.subplots(2,4)
ar[0,0].imshow(images_with_message[0])
ar[0,1].imshow(images_with_message[1])
ar[0,2].imshow(images_with_message[2])
ar[0,3].imshow(images_with_message[3])
ar[1,0].imshow(images_with_message[4])
ar[1,1].imshow(images_with_message[5])
ar[1,2].imshow(images_with_message[6])
ar[1,3].imshow(images_with_message[7])

In [ ]:
errors = [mse(im, image) for im in images_with_message]
x = np.arange(1, 9, 1)
y = np.array(errors)
plt.xlabel('nbits')
plt.ylabel('Mean Square Error')
plt.plot(x, y)

# Zadanie 3

In [ ]:
def hide_message_upgraded(image, message, nbits=1, spos=0):
    """Hide a message in an image (LSB).
    
    nbits: number of least significant bits
    """
    nbits = clamp(nbits, 1, 8)
    shape = image.shape
    image = np.copy(image).flatten()
    if len(message) > len(image) * nbits:
        raise ValueError("Message is to long :(")
    if spos + len(message) > len(image) * nbits:        # Added error
        raise ValueError("Message won't fit in image")
    
    chunks = [message[i:i + nbits] for i in range(0, len(message), nbits)]
    for i, chunk in enumerate(chunks):
        byte = "{:08b}".format(image[i+spos*8])     # Change
        new_byte = byte[:-nbits] + chunk
        image[i+spos*8] = int(new_byte, 2)         # Change
        
    return image.reshape(shape)


def reveal_message_upgraded(image, nbits=1, length=0, spos=0):
    """Reveal the hidden message.
    
    nbits: number of least significant bits
    length: length of the message in bits.
    """
    nbits = clamp(nbits, 1, 8)
    shape = image.shape
    image = np.copy(image).flatten()
    length_in_pixels = math.ceil(length/nbits)
    if len(image) < length_in_pixels or length_in_pixels <= 0:
        length_in_pixels = len(image)
    
    message = ""
    i = spos*8         # Change
    while i < length_in_pixels:
        byte = "{:08b}".format(image[i])
        message += byte[-nbits:]
        i += 1
        
    mod = length % -nbits
    if mod != 0:
        message = message[:mod]
    return message

# Read message from exercise 1 but start from 4th character
unknown_message = decode_from_binary_array(reveal_message_upgraded(unknown_image, nbits=n, length=len(message)*8, spos=4))
print(unknown_message)

# Hide message from exercise 1 but start from 3th byte of image
image_with_message = hide_message_upgraded(image, encode_as_binary_array(message), n, spos=3)
#  and then show message as it starts from 0 byte - first 3 bytes won't make much sense
unknown_message = decode_from_binary_array(reveal_message(image_with_message, nbits=n, length=len(message)*8))
print(unknown_message)

# Zadanie 4

In [ ]:
def get_hidden_image(image, hidden_nbits, hidden_length):
    hidden_image = reveal_message(image_with_secret, nbits=hidden_nbits, length=hidden_length)
    hidden_image = int(hidden_image, 2).to_bytes((len(hidden_image) + 7) // 8, byteorder='big')
    
    return hidden_image
image_with_secret = load_image("images/image_in_image.png")
with open("images/hidden.jpg","wb") as f:
    f.write(get_hidden_image(image_with_secret, 1, 191568))
plt.imshow(load_image('images\hidden.jpg'))

# Zadanie 5

In [ ]:
def get_hidden_image_upgrade(image, nbits):
    nbits = clamp(nbits, 1, 8)
    shape = image.shape
    image = np.copy(image).flatten()
    footer_string = 'ffd9'
    footer, last_4bytes = "{0:08b}".format(int(footer_string, 16)), "0000000000000000"
    message = ""
    i = 0
    while i < len(image):
        byte = "{:08b}".format(image[i])
        message += byte[-nbits:]
        last_4bytes = last_4bytes[nbits:] + byte[-nbits:]
        if last_4bytes == footer:
            break
        i += 1

    hidden_image = int(message, 2).to_bytes((len(message) + 7) // 8, byteorder='big')
    return hidden_image

image_with_secret = load_image("images/image_in_image.png")
secret = get_hidden_image_upgrade(image_with_secret, 1)

nparr = np.frombuffer(secret, np.uint8)
secret_image = cv.imdecode(nparr, cv.IMREAD_COLOR)
plt.imshow(cv.cvtColor(secret_image, cv.COLOR_BGR2RGB))